In [1]:
import numpy as np
import pandas as pd
import requests
import config
import os
import json
from pandas.io.json import json_normalize
import pygsheets

##Schedule_a API guide: https://api.open.fec.gov/developers/#/receipts/get_schedules_schedule_a_

In [2]:
### MULTI-CANDIDATE SEARCH ###
#Set search for all 2020 (two-year transaction period) Maine Congressional candidates
cand_state = 'ME'
period='2020'
parameters = {'election_year':period
            ,'state':cand_state
            ,'api_key':config.api_key}
#To iterate over comm_ids
id=int(0)

#Requests candidate info
r_cands = requests.get('https://api.open.fec.gov/v1/candidates/search',params=parameters).json()

#Locates and sets Committee ID from 'principal_committees' sub-array
#Output list of IDs
comm_ids=json_normalize(data=r_cands['results'],record_path='principal_committees')['committee_id'].tolist()
comm_ids

['C00715532',
 'C00713958',
 'C00710087',
 'C00314575',
 'C00709899',
 'C00653816',
 'C00703413',
 'C00706739',
 'C00683193',
 'C00433391',
 'C00709279',
 'C00712323',
 'C00694364']

In [4]:
#Print committee list to validate
print(json_normalize(data=r_cands['results'],record_path='principal_committees')[['committee_id','name','party']])

   committee_id                             name party
0     C00710087         MICHAEL BUNKER FOR MAINE   DEM
1     C00314575              COLLINS FOR SENATOR   REP
2     C00709899            SARA GIDEON FOR MAINE   DEM
3     C00653816  COMMITTEE TO ELECT JARED GOLDEN   DEM
4     C00703413            FRIENDS OF BRE KIDMAN   DEM
5     C00706739            DEREK FOR U.S. SENATE   REP
6     C00683193      DR CAT LONDON FOR US SENATE   DEM
7     C00433391             PINGREE FOR CONGRESS   DEM
8     C00709279                 SWEET FOR SENATE   DEM
9     C00712323       JON TREACY FORWARD 4 MAINE   DEM
10    C00694364   DANIELLE VANHELSING FOR SENATE   IND


In [3]:
#Initialize dataframe collector for itemized contribs
dfs=[]
id=int(0)

#Loop through pages
for x in range(len(comm_ids)):
    
# print('FOR')
    #Initialize query dict
    querydict = {'per_page':'100'
                  ,'committee_id':comm_ids[id]
                  ,'sort':'contribution_receipt_date'
                  ,'api_key':config.api_key
                  ,'is_individual':'true'
                  ,'two_year_transaction_period':period
                 }

    r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=querydict).json()
    
    #Results rows
    df = json_normalize(r['results'])
    dfs.append(df)
    
    #Last page variables
    if r['pagination']['last_indexes'] is None:
        None
        if id<len(comm_ids[0]):
                id=id+1
        else:
            break
    else: 
        last_index=pd.to_numeric(r['pagination']['last_indexes']['last_index'])
        last_date=r['pagination']['last_indexes']['last_contribution_receipt_date']
        
        #Loop through pages
        while id<len(comm_ids[0]):

#print('WHILE')

            #Initialize query, without last_index and date
            querydict.update([('last_index',last_index)
                            ,('last_contribution_receipt_date',last_date)])

            r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=querydict).json()

            #Results rows
            df = json_normalize(r['results'])
            dfs.append(df)

            #Update Pagination variables
            last_index=pd.to_numeric(r['pagination']['last_indexes']['last_index'])
            last_date=r['pagination']['last_indexes']['last_contribution_receipt_date']        
            dfs.append(df)
            
            #Increment to max ID in list
            if id<len(comm_ids[0]):
                id=id+1
            else:
                break

#Concatenate all dfs
itemdf=pd.concat(dfs,sort=False)
itemdf=itemdf.drop_duplicates(subset='transaction_id')

In [4]:
#Itemized dataframe query
itemdf_c=itemdf[['committee.candidate_ids'
                ,'committee.city'
                ,'committee.committee_type_full'
                ,'committee.cycle'
                ,'committee.name'
                ,'committee.party_full'
                ,'committee.state_full'
                ,'contribution_receipt_amount'
                ,'contribution_receipt_date'
                ,'contributor_aggregate_ytd'
                ,'contributor_city'
                ,'contributor_employer'
                ,'contributor_first_name'
                ,'contributor_last_name'
                ,'contributor_name'
                ,'contributor_occupation'
                ,'contributor_state'
                ,'contributor_street_1'
                ,'contributor_zip'
                ,'entity_type_desc'
                ,'fec_election_type_desc'
                ,'fec_election_year'
                ,'line_number_label'
                ,'load_date'
                ,'pdf_url'
                ,'transaction_id']]

itemdf_c

,committee.candidate_ids,committee.city,committee.committee_type_full,committee.cycle,committee.name,committee.party_full,committee.state_full,contribution_receipt_amount,contribution_receipt_date,contributor_aggregate_ytd,...,contributor_state,contributor_street_1,contributor_zip,entity_type_desc,fec_election_type_desc,fec_election_year,line_number_label,load_date,pdf_url,transaction_id
0,[S6ME00159],BANGOR,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,250.0,2019-01-01T00:00:00,750.0,...,FL,15019 MADEIRA WAY,337387060,INDIVIDUAL,PRIMARY,2020,Contributions From Individuals/Persons Other T...,2019-04-09T02:42:36.656000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,A1F99550E176249C7AFC
1,[S6ME00159],BANGOR,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,100.0,2019-01-01T00:00:00,500.0,...,AL,4114 CITATION CIR,354062944,INDIVIDUAL,PRIMARY,2020,Contributions From Individuals/Persons Other T...,2019-04-09T02:42:36.677000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,A66BF5ABC54F943FA94E
2,[S6ME00159],BANGOR,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,100.0,2019-01-01T00:00:00,400.0,...,AR,16735 OLD HIGHWAY 68,727618641,INDIVIDUAL,PRIMARY,2020,Contributions From Individuals/Persons Other T...,2019-04-09T02:42:36.764000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,AE9C7DC4C7855441A843
3,[S6ME00159],BANGOR,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,100.0,2019-01-01T00:00:00,300.0,...,FL,953 HILLSBORO MILE,330622501,INDIVIDUAL,PRIMARY,2020,Contributions From Individuals/Persons Other T...,2019-04-09T02:42:36.843000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,AB3531B9E6A664583BE4
4,[S6ME00159],BANGOR,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,100.0,2019-01-01T00:00:00,200.0,...,WA,10750 OGLE RD NE,983706907,INDIVIDUAL,PRIMARY,2020,Contributions From Individuals/Persons Other T...,2019-04-09T02:42:36.852000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,A8BDD8FFE4FE34C7DAA7
5,[S6ME00159],BANGOR,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,100.0,2019-01-01T00:00:00,200.0,...,WA,10750 OGLE RD NE,983706907,INDIVIDUAL,PRIMARY,2020,Contributions From Individuals/Persons Other T...,2019-04-09T02:42:36.852000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,AA2E9430417814DCAB3B
6,[S6ME00159],BANGOR,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,100.0,2019-01-01T00:00:00,400.0,...,TX,4412 LARCHMONT ST,752051621,INDIVIDUAL,PRIMARY,2020,Contributions From Individuals/Persons Other T...,2019-04-09T02:42:36.867000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,A0950552B2BCE4148A89
7,[S6ME00159],BANGOR,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,100.0,2019-01-02T00:00:00,300.0,...,CA,15965 LANCASTER RD,950303085,INDIVIDUAL,PRIMARY,2020,Contributions From Individuals/Persons Other T...,2019-04-09T02:42:36.769000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,AB2FD1F7334D440D0917
8,[S6ME00159],BANGOR,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,250.0,2019-01-02T00:00:00,250.0,...,MS,8051 CRESTWOOD CV,395328383,INDIVIDUAL,PRIMARY,2020,Contributions From Individuals/Persons Other T...,2019-04-09T02:42:36.838000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,ADFE3250EAA894F028D3
9,[S6ME00159],BANGOR,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,50.0,2019-01-03T00:00:00,250.0,...,FL,309 NW 24TH ST,326072693,INDIVIDUAL,PRIMARY,2020,Contributions From Individuals/Persons Other T...,2019-04-09T02:42:36.744000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,AC9527181AD7C477EB4D


In [5]:
#Write itemized individual results to local CSV
cwd = os.getcwd()
itemdf.to_csv(cwd+'/data/maine-fed-itemized-receipts.csv')

In [8]:
#Write itemized individual results to Google Sheet

#Google Credentials
gc = pygsheets.authorize(service_file=cwd+'/me-congress-2020-creds.json')

#Select sheet and worksheet
sh = gc.open('me-congress-2020')
#sh = gc.open_by_key('1AKrgHT9NLpoddV16B7_M_0PEjJmMQAGtXJUnLCTDHjA')
wks = sh[0]

#Clear sheet before load
wks.clear(start='A1',fields='*')

#Write contribs dataframe to sheet
wks.set_dataframe(itemdf_c,(1,1))

In [12]:
#Initialize summary stat query
p=int(1)

querydict_u = {'per_page':'20'
               ,'api_key':config.api_key
               ,'cycle':'2020'
               ,'page':p
               ,'sort_nulls_last':'false'
               ,'sort':'-cycle'
               ,'sort_hide_null':'false'
                 }

In [13]:
#New dfs collector
dfs_u=[]
id_u=int(0)

#Loop through pages
for x in range(len(comm_ids)):
    
    print('FOR')
    p=int(1)

    r_u = requests.get('https://api.open.fec.gov/v1/committee/'+comm_ids[id_u]+'/totals/',params=querydict_u).json()
        
    #Loop through pages
    while p <= r_u['pagination']['pages']:

        print('WHILE')

        #Results rows
        df_u = json_normalize(r_u['results'])
        dfs_u.append(df_u)

        p=p+1
        r_u = requests.get('https://api.open.fec.gov/v1/committee/'+comm_ids[id_u]+'/totals/',params=querydict_u).json()

    #Increment to next candidate
    df_u = json_normalize(r_u['results'])
    dfs_u.append(df_u)
    id_u=id_u+1


#Concatenate all dfs
unitemdf=pd.concat(dfs_u,sort=False)
unitemdf=unitemdf.drop_duplicates(subset='committee_id')
unitemdf

FOR
FOR
WHILE
FOR
FOR
WHILE
FOR
WHILE
FOR


KeyboardInterrupt: 

{'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz',
 'cycle': '2020',
 'page': 1,
 'per_page': '20',
 'sort': '-cycle',
 'sort_hide_null': 'false',
 'sort_nulls_last': 'false'}

In [11]:
#Write itemized individual results to Google Sheet

#Google Credentials
gc = pygsheets.authorize(service_file=cwd+'/me-congress-2020-creds.json')

#Select sheet and worksheet
sh = gc.open('me-congress-2020')
#sh = gc.open_by_key('1AKrgHT9NLpoddV16B7_M_0PEjJmMQAGtXJUnLCTDHjA')
wks_u = sh[1]

#Clear sheet before load
wks_u.clear(start='A1',fields='*')

#Write contribs dataframe to sheet
wks_u.set_dataframe(unitemdf,(1,1))